In [117]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import re

from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression

from sklearn.model_selection import (cross_val_score, train_test_split, 
                                     KFold, GridSearchCV)

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Movie Recommendation Engine - Content based filtering using Cosine Similarity Matrix 

In [125]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
genres_df = pd.read_csv('genres.csv')
print(movies_df.info())
imdb_df = pd.read_csv('IMDB-Movie-Data.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
None


In [126]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [102]:
# movies_df['genres'] = movies_df['genres'].apply(lambda x: x.split()).apply(lambda y: '| '.join(y))
# movies_df['genres'] = movies_df['genres'].apply(lambda x: x.split('|'))

In [103]:
# imdb_df['Genre'] = imdb_df['Genre'].apply(lambda x: x.split()).apply(lambda y: ', '.join(y))
# imdb_df['Genre'] = imdb_df['Genre'].apply(lambda x: x.split(','))

In [280]:
imdb_df

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,Movie_id,imp_features
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,1,"James Gunn,Chris Pratt, Vin Diesel, Bradley Co..."
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,2,"Ridley Scott,Noomi Rapace, Logan Marshall-Gree..."
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0,3,"M. Night Shyamalan,James McAvoy, Anya Taylor-J..."
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0,4,"Christophe Lourdelet,Matthew McConaughey,Reese..."
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0,5,"David Ayer,Will Smith, Jared Leto, Margot Robb..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,NaN,45.0,996,"Billy Ray,Chiwetel Ejiofor, Nicole Kidman, Jul..."
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,17.54,46.0,997,"Eli Roth,Lauren German, Heather Matarazzo, Bij..."
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,58.01,50.0,998,"Jon M. Chu,Robert Hoffman, Briana Evigan, Cass..."
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,NaN,22.0,999,"Scot Armstrong,Adam Pally, T.J. Miller, Thomas..."


In [281]:
imdb_df.shape

(1000, 14)

In [282]:
# Columns for recommendation engine
columns = ['Actors', 'Director', 'Genre', 'Title']

In [283]:
imdb_df[columns]

,Actors,Director,Genre,Title
0,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",James Gunn,"Action,Adventure,Sci-Fi",Guardians of the Galaxy
1,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",Ridley Scott,"Adventure,Mystery,Sci-Fi",Prometheus
2,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",M. Night Shyamalan,"Horror,Thriller",Split
3,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",Christophe Lourdelet,"Animation,Comedy,Family",Sing
4,"Will Smith, Jared Leto, Margot Robbie, Viola D...",David Ayer,"Action,Adventure,Fantasy",Suicide Squad
...,...,...,...,...
995,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",Billy Ray,"Crime,Drama,Mystery",Secret in Their Eyes
996,"Lauren German, Heather Matarazzo, Bijou Philli...",Eli Roth,Horror,Hostel: Part II
997,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",Jon M. Chu,"Drama,Music,Romance",Step Up 2: The Streets
998,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",Scot Armstrong,"Adventure,Comedy",Search Party


In [ ]:
imdb_df['Title'] 

In [314]:
title = 'The Imitation Game'
imdb_df[imdb_df.Title == title][columns].values[0]

array(['Benedict Cumberbatch, Keira Knightley, Matthew Goode, Allen Leech',
       'Morten Tyldum', 'Biography,Drama,Thriller', 'The Imitation Game'],
      dtype=object)

In [315]:
imdb_df[imdb_df.Title == 'The Girl with the Dragon Tattoo'][columns].values[0]

array(['Daniel Craig, Rooney Mara, Christopher Plummer,Stellan Skarsgård',
       'David Fincher', 'Crime,Drama,Mystery',
       'The Girl with the Dragon Tattoo'], dtype=object)

In [316]:
imdb_df[imdb_df.Title == 'The Proposal'][columns].values[0]

array(['Sandra Bullock, Ryan Reynolds, Mary Steenburgen,Craig T. Nelson',
       'Anne Fletcher', 'Comedy,Drama,Romance', 'The Proposal'],
      dtype=object)

In [330]:
imdb_df[imdb_df.Title == 'The Visit'][columns].values[0] 

array(['Olivia DeJonge, Ed Oxenbould, Deanna Dunagan, Peter McRobbie',
       'M. Night Shyamalan', 'Comedy,Horror,Thriller', 'The Visit'],
      dtype=object)

In [331]:
imdb_df[imdb_df.Title == 'Gone Girl'][columns].values[0] 

array(['Ben Affleck, Rosamund Pike, Neil Patrick Harris,Tyler Perry',
       'David Fincher', 'Crime,Drama,Mystery', 'Gone Girl'], dtype=object)

In [284]:
imdb_df[columns].isnull().values.any()

False

In [285]:
# Function to combine values of these columns
def get_imp_features(data):
    imp_features = []
    for i in range(0, data.shape[0]):
        imp_features.append(data['Title'][i]+","+data['Actors'][i]+","+data['Genre'][i]+","+data['Director'][i])
        
    return imp_features

In [286]:
imdb_df['imp_features'] = get_imp_features(imdb_df)

In [287]:
imdb_df['imp_features'][0]

'Guardians of the Galaxy,Chris Pratt, Vin Diesel, Bradley Cooper, Zoe Saldana,Action,Adventure,Sci-Fi,James Gunn'

In [326]:
vectorizer = CountVectorizer()
cm = vectorizer.fit_transform(imdb_df['imp_features'])
vectorizer.get_feature_names_out()

In [328]:
vectorizer.get_feature_names_out()

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names_out'

In [296]:
cm = CountVectorizer().fit_transform(imdb_df['imp_features'])
cm

<1000x4645 sparse matrix of type '<class 'numpy.int64'>'
	with 15329 stored elements in Compressed Sparse Row format>

In [298]:
# Get the Cosine Similarity Matrix from count matrix cm
cs = cosine_similarity(cm)
print(cs)

[[1.         0.1767767  0.06085806 ... 0.0571662  0.06537205 0.        ]
 [0.1767767  1.         0.         ... 0.         0.06933752 0.        ]
 [0.06085806 0.         1.         ... 0.         0.         0.        ]
 ...
 [0.0571662  0.         0.         ... 1.         0.06726728 0.        ]
 [0.06537205 0.06933752 0.         ... 0.06726728 1.         0.07161149]
 [0.         0.         0.         ... 0.         0.07161149 1.        ]]


In [305]:
# Shape of Cosine Similarity Matrix (rows x rows)
cs.shape

(1000, 1000)

In [306]:
# title = 'Gone Girl'
title = 'The Imitation Game'
movie_id = imdb_df[imdb_df.Title == title]['Movie_id'].values[0]

# Create list of enumerations for the similarity scores [(movie_id,similarity_score)]

In [307]:
scores = list(enumerate(cs[movie_id]))

In [308]:
sorted_similarity_scores = sorted(scores, key=lambda x:x[1], reverse = True) # x[0] --> movie_id; x[1] --> similarity score

In [312]:
sorted_similarity_scores = sorted_similarity_scores[1:]
sorted_similarity_scores

[(224, 0.30316953129541624),
 (457, 0.2886751345948129),
 (134, 0.2672612419124244),
 (372, 0.2672612419124244),
 (84, 0.2581988897471611),
 (319, 0.2581988897471611),
 (668, 0.2581988897471611),
 (95, 0.25),
 (704, 0.25),
 (438, 0.23570226039551587),
 (284, 0.20801257358446093),
 (13, 0.2004459314343183),
 (248, 0.2004459314343183),
 (288, 0.2004459314343183),
 (386, 0.2004459314343183),
 (571, 0.2004459314343183),
 (716, 0.2004459314343183),
 (33, 0.19364916731037082),
 (161, 0.19364916731037082),
 (210, 0.19364916731037082),
 (298, 0.19364916731037082),
 (341, 0.19364916731037082),
 (600, 0.19364916731037082),
 (663, 0.19364916731037082),
 (702, 0.19364916731037082),
 (219, 0.1875),
 (272, 0.1875),
 (276, 0.1875),
 (469, 0.1875),
 (660, 0.1875),
 (15, 0.18190171877724973),
 (878, 0.18190171877724973),
 (803, 0.1636634176769943),
 (790, 0.14433756729740646),
 (394, 0.1386750490563073),
 (722, 0.1386750490563073),
 (846, 0.1386750490563073),
 (937, 0.1386750490563073),
 (52, 0.1336306

In [313]:
j=0
print('Movie Recommendation Engine - Content based filtering using Cosine Similarity Matrix \n \n')
print('If you liked the movie: ', title, ', check out the following 10 movies: \n' )
for item in sorted_similarity_scores:
    recommended_movie = imdb_df[imdb_df.Movie_id == item[0]]['Title'].values[0]
    print(j+1, '.', recommended_movie)
    j=j+1
    if j>9:
        break

Movie Recommendation Engine - Content based filtering using Cosine Similarity Matrix 
 

If you liked the movie:  The Imitation Game , check out the following 10 movies: 

1 . The Girl with the Dragon Tattoo
2 . The Proposal
3 . Whiplash
4 . The Maze Runner
5 . Gone Girl
6 . The Visit
7 . Ted 2
8 . Avengers: Age of Ultron
9 . Australia
10 . The imposible
